In [4]:
""" from datasets import load_dataset
dataset = load_dataset("wmt17", "de-en", split="train[:1%]") 


for example in dataset['translation']:
    print(example['de']) """

' from datasets import load_dataset\ndataset = load_dataset("wmt17", "de-en", split="train[:1%]") \n\n\nfor example in dataset[\'translation\']:\n    print(example[\'de\']) '

In [5]:

from torch.utils.data import DataLoader
from datasets import load_dataset
from src.utils.data_cleaning import clean_dataset
from src.modelling.tokenizer import CustomBPETokenizer
from src.dataset import TranslationDataset

# Load the dataset
dataset = load_dataset("wmt17", "de-en", split="train[:1%]") 
cleaned_data = clean_dataset(dataset)
# Initialize and use the tokenizer



/Users/hanneswidera/Uni/Master/Semester 3/Transformer/transformer/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:

tokenizer = CustomBPETokenizer(cleaned_data, vocab_size=50000, max_length=64)

In [7]:
print(tokenizer.gpt2_tokenizer)

GPT2Tokenizer(name_or_path='/Users/hanneswidera/Uni/Master/Semester 3/Transformer/transformer/storage/gpt2_from_bpe', vocab_size=50000, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	50000: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}


In [8]:
tokenizer.convert_tokens_to_ids("[BOS]")

1

In [9]:
# Test the tokenizer 
def tokenizer_test(example):
    tokens = tokenizer.tokenize(example)
    print("Tokens:", tokens)

    encoded = tokenizer.encode(example)
    print("Encoded:", encoded)

    decoded = tokenizer.decode(encoded)
    print("Decoded:", decoded)
    
    
for example in cleaned_data[:2]:
    print(example)
    tokenizer_test(example['src'])
    print("_______")
    tokenizer_test(example['tgt'])

{'de': 'Ich erkläre die am Freitag, dem 17. Dezember unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen, wünsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe, daß Sie schöne Ferien hatten.', 'en': 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.', 'src': 'ich erkläre die am freitag, dem 17. dezember unterbrochene sitzungsperiode des europäischen parlaments für wiederaufgenommen, wünsche ihnen nochmals alles gute zum jahreswechsel und hoffe, daß sie schöne ferien hatten.', 'tgt': 'i declare resumed the session of the european parliament adjourned on friday 17 december 1999, and i would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.'}
Tokens: ['ich', 'ĠerklÃ¤re', 'Ġdie', 'Ġam', 'Ġfreitag', ',', 'Ġdem', 'Ġ17', '.', 'Ġdezember', 'Ġunterb', 'r

In [16]:
from src.dataset import TranslationDataset

translation_dataset = TranslationDataset(cleaned_data, tokenizer)

dataloader = DataLoader(translation_dataset, batch_size=32, shuffle=True)

first_batch = next(iter(dataloader))
print(first_batch)

{'src_input_ids': tensor([[   1,   78,  126,  ...,    0,    0,    0],
        [   1,  407, 1426,  ...,    0,    0,    0],
        [   1,   42,  387,  ...,    0,    0,    0],
        ...,
        [   1,  252,   77,  ...,    0,    0,    0],
        [   1,  268,  395,  ...,    0,    0,    0],
        [   1,  252,   77,  ...,    0,    0,    0]]), 'tgt_input_ids': tensor([[   78,   126,   806,  ...,     0,     0,     0],
        [  407,  1426,   186,  ...,     0,     0,     0],
        [   42,   387,   149,  ...,     0,     0,     0],
        ...,
        [  252,    77,   126,  ...,     0,     0,     0],
        [  268,   395,  1052,  ...,     0,     0,     0],
        [  252,    77, 29336,  ...,     0,     0,     0]])}


# Positional Encoding + Embedding layer

In [17]:
# Example usage
import torch
from src.modelling.positional_encoding import PositionalEncoding
from src.modelling.word_embedding import WordEmbedding


vocab_size = 50000
d_model = 512
max_len = 64

# Initialize layers
word_embedding = WordEmbedding(vocab_size, d_model)
positional_encoding = PositionalEncoding(d_model, max_len)

# Example input (batch_size, sequence_length)
input_ids = torch.randint(0, vocab_size, (32, max_len))

# Apply word embedding and positional encoding
embedded = word_embedding(input_ids)
encoded = positional_encoding(embedded)

print(encoded.shape)  # Output: torch.Size([32, 100, 512])

torch.Size([32, 64, 512])


In [ ]:
# Iterate over the DataLoader to get batches of data
for batch in dataloader:
    input_ids = batch["src_input_ids"]
    target_ids = batch["tgt_input_ids"]

    # Apply word embedding and positional encoding
    embedded = word_embedding(input_ids)
    encoded = positional_encoding(embedded)

    # batchsize x max_length x embedding size
    print(encoded.shape)  # Output: torch.Size([32, 64, 512])
    break  # Remove this break to process all batches

torch.Size([32, 64, 512])
